In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_psy = pd.read_csv('psychologists_data.csv')
df_psy.shape

(877, 21)

У нас там много параметров, которые в себя включают списки. Кажется, такое немного неудобно анализировать. Из них хотим сделать столбцы с бинарными переменными (работа в подходе i = да или нет,  работает с детьми/взрослыми/пожилыми - да или нет и так далее). Да, параметров станет не 21 а 201, ну ниче ниче.... Допустим, мы все делаем правильно...

<a href="https://ibb.co/jvy6fdZZ"><img src="https://i.ibb.co/yn6fPDBB/IMG-5595.jpg" alt="IMG-5595" border="0"></a>

In [ ]:
#все массивы которые нужно преобразовать в столбцы
problems = []
for a in df_psy['Специализация']:
  for el in a[1:-1].split("', '"):
    problems.append(el.strip("'"))
problems = list(set(problems))

approaches = []
for a in df_psy['Подходы']:
  for el in a[1:-1].split("', '"):
    approaches.append(el.strip("'"))
approaches = list(set(approaches))

works_with =  ['Взрослые', 'Дети от 3 до 10 лет', 'Подростки от 11 до 15 лет', 'Подростки от 16 до 18 лет',  'Семьи', 'Люди в возрасте']

In [ ]:
df = df_psy.copy()

In [ ]:
#как будем массивы в колонки преобразовывать
def transform_to_columns(df, list, word, column):
    for i in list:
        df[f'{word}: {i}'] = df[column].apply(lambda x: 1 if i in x else 0)
    return df

Если делать все одним датафреймом, то он получается ужасно длинный. Разделим на три.

**Добавим новые признаки** -- ширина подхода (сколько подходов использует психолог), ширину специализации (со сколькими запросами к нему можно обращаться) и ширину возрастных групп (какие возрастные группы могут обратиться в терапию)

In [ ]:
#создали отдельный датафрейм с подходами и зафигачили туда широту подходов
df_appr = df_psy[['Имя фамилия', 'Цена за сессию (руб)', 'Подходы']]
df_appr = transform_to_columns(df_appr, approaches, 'Подход', 'Подходы')
df_appr['Широта подхода'] = df_appr.iloc[:, 3:-1].fillna(0).sum(axis=1)

Дропнем колонку "Подходы", она больше не нужна


In [ ]:
#создали отдельный датафрейм со специализациями и зафигачили туда широту подходов
df_spec = df[['Имя фамилия', 'Цена за сессию (руб)', 'Специализация']]
df_spec = transform_to_columns(df_spec, problems, 'Запрос', 'Специализация')
df_spec['Широта специализации'] = df_spec.iloc[:, 3:-2].fillna(0).sum(axis=1)
df_spec.head()

,Имя фамилия,Цена за сессию (руб),Специализация,Запрос: Коучинг руководителей,Запрос: Сексуальные и гендерные вопросы,Запрос: Страхи,Запрос: Профориентация,Запрос: Трудные жизненные ситуации,Запрос: Психосоматика,Запрос: Work life balance,...,Запрос: Синдром Аспергера,Запрос: Работа с эмоциями,Запрос: Поведение ребенка,Запрос: Зависимость и созависимость в отношениях,Запрос: Отношения с детьми,Запрос: Потеря близкого,Запрос: Коммуникативные навыки,Запрос: Зависимости,Запрос: Разводы и расставания,Широта специализации
0,Земфира Максимова,3500,"['Самооценка и уверенность в себе', 'Сложные п...",0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,13
1,Алина Иванова,5000,"['Самооценка и уверенность в себе', 'Страхи', ...",0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,10
2,Светлана Матякубова,5000,"['Самооценка и уверенность в себе', 'Трудный п...",0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,10
3,Екатерина Новицкая,5000,"['Самооценка и уверенность в себе', 'Страхи', ...",0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,8
4,Елена Щеблыкина,4000,"['Самооценка и уверенность в себе', 'Эмоционал...",0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,9


In [ ]:
df_appr.drop('Подходы', axis = 1, inplace = True)
df_spec.drop(['Специализация'], axis = 1, inplace = True)

In [ ]:
df_psy = transform_to_columns(df_psy, works_with, 'Группа', 'С кем работает')

In [ ]:
df_psy['Широта возрастных групп'] = df_psy.iloc[:, -6:-1].sum(axis = 1)
df_psy.drop(['Подходы', 'Специализация', 'С кем работает'], axis = 1, inplace = True)
df_psy.head()

,Имя фамилия,Цена за сессию (руб),Город,Онлайн,Лично,Опыт работы (в годах),Возраст (в годах),Семейное положение,Пол,Наличие детей,...,Наличие видеоприветствия,Наличие аудиоприветствия,О себе,Группа: Взрослые,Группа: Дети от 3 до 10 лет,Группа: Подростки от 11 до 15 лет,Группа: Подростки от 16 до 18 лет,Группа: Семьи,Группа: Люди в возрасте,Широта возрастных групп
0,Земфира Максимова,3500,Калининград,1,1,4,45.0,0.0,0,0.0,...,1,0,"Здравствуйте меня зовут Земфира, я психолог, г...",1,0,0,0,0,0,1
1,Алина Иванова,5000,Москва,1,0,7,34.0,1.0,0,1.0,...,0,1,"Верю в силы, возможности и способности человек...",1,0,0,0,0,0,1
2,Светлана Матякубова,5000,Санкт-Петербург,1,1,17,57.0,1.0,0,1.0,...,0,0,"Я психолог, психоаналитической ориентации, спе...",1,0,0,0,0,0,1
3,Екатерина Новицкая,5000,Москва,1,0,5,29.0,1.0,0,0.0,...,1,0,"Здравствуйте! Меня зовут Екатерина, я психолог...",1,0,0,0,0,0,1
4,Елена Щеблыкина,4000,Томск,1,0,5,28.0,0.0,0,0.0,...,0,0,В своей практике опираюсь на проверенные метод...,1,0,0,0,0,0,1


Создадим новую переменную - длина текста "О себе".

In [ ]:
df_psy['Длина "О себе"'] = df_psy["О себе"].apply(len)

Смотрим на дубликаты и пропуски в наших датафреймах:

In [ ]:
df_psy.duplicated().sum()

0

In [ ]:
df_psy.isnull().sum()

,0
Имя фамилия,0
Цена за сессию (руб),0
Город,0
Онлайн,0
Лично,0
Опыт работы (в годах),0
Возраст (в годах),29
Семейное положение,138
Пол,0
Наличие детей,133


Есть пропуски только в графах "Семейное положение", "наличие детей" и "возраст". Сделаем с ними что-нибудь:

Заполним возраст средним значением;

Наличие детей заполним нулями, так как при отсутсвии указания на обратное, специалист вряд ли захочет/сможет поднимать вопросы, связанные с материнством и отцовством, а так же не попадется в поиске людям, которые ищут специалиста с детьми. (другими словами, будем считать, что детей нет),

Нули в семейном положении обозначим новым значением "Не указано"

In [ ]:
df_psy['Возраст (в годах)'] = df_psy['Возраст (в годах)'].fillna(df_psy['Возраст (в годах)'].mean())
df_psy['Семейное положение'] = df_psy['Семейное положение'].fillna(2) #новое значение, означает -- не указано
df_psy['Наличие детей'] = df_psy['Наличие детей'].fillna(0) #если не указано -- будем считать, что детей нет

In [ ]:
df_psy.isnull().sum()

,0
Имя фамилия,0
Цена за сессию (руб),0
Город,0
Онлайн,0
Лично,0
Опыт работы (в годах),0
Возраст (в годах),0
Семейное положение,0
Пол,0
Наличие детей,0


Видно, что не все подходы и специализации используются одинаково. Какие-то подходы использует очень мало психологов. Исключим из анализа такие подходы, которые использует меньше 1% психологов.

In [ ]:
cnt_appr = df_appr[df_appr.columns[2:-1]].sum()
div = int(879 * 0.01)

little_appr = cnt_appr[cnt_appr < div]
little_appr

,0
Подход: Символдрама,6
Подход: Фокусинг,7
Подход: MBCT,7
Подход: Нейропсихология,4


Аналогично со специализациями. Мы их не исключаем полностью из анализа, но возможно будем отдельно с ними что-то думать. (Спойлер: не придумали. Из анализа исключать не стали, но и серьезных зависимостей с ними не уловили)

In [ ]:
cnt_spec = df_spec[df_spec.columns[2:-2]].sum()
div = int(879 * 0.01)

little_spec = cnt_spec[cnt_spec < div]
little_spec

,0
Запрос: Коучинг руководителей,1
Запрос: Work life balance,3
Запрос: Карьерные изменения,1
Запрос: Самодисциплина,1
Запрос: РАС,4
Запрос: СДВГ у детей,1
Запрос: Синдром Аспергера,2


In [ ]:
df_spec.to_csv('df_spec.csv', index= False)
df_appr.to_csv('df_appr.csv', index= False)
df_psy.to_csv('df_psy.csv', index = False)

## Встретимся в EDA :D

<a href="https://ibb.co/1YFk7p5K"><img src="https://i.ibb.co/twV9QNtc/image.png" alt="image" border="0"></a>